# -----
# FIG 8 in RECCAP SO paper -> compute trends for different time periods
# -----
# trends in simB are loaded from netcdf file produced by: PAPER_RECCAPv2_SO_Suppl_time_series_trends_simB.ipynb
#
# Plot time series of CO2 fluxes 
# drift-corrected
# 
# Contact: cara.nissen@awi.de or cara.nissen@colorado.edu
#
# version: August 2023
#

In [2]:
### modules
import os
import numpy as np
import seawater as sw
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm 
import matplotlib.gridspec as gridspec
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from netCDF4 import Dataset
from datetime import date, timedelta
import copy
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature
from annualmean import annualmean
from datetime import date, timedelta
from sklearn.linear_model import LinearRegression
from mpl_toolkits.axes_grid.inset_locator import inset_axes

/tmp/ipykernel_1528226/1677564082.py:23: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import inset_axes


In [3]:
#-----
# SETTINGS
# define paths to all data
# define where to save plots (if any)
# define years to average over
#-----

# data sets
path1 = '/pscratch/sd/c/cnissen/RECCAPv2/'
path_models = path1+'reccap_submissions/download_20220124/Models/2D_CO2/' 
path_data   = path1+'reccap_submissions/download_20220124/Surface_CO2/'
path_atminv = path1+'reccap_submissions/download_20220124/Atmospheric_inversions/'
path_soccom = path1+'reccap_submissions/download_20220124/Surface_CO2/'
path_trend= path1+'reccap_submissions/download_20220124/Models/Linear_trends/' 

# river flux adjustment
path_river  = path1+'masks_reccap/river_flux_adjustment/'

# path to RECCAP SO mask
path_mask = path1+'masks_reccap/'

#----
# specify years to average over
#----
# NOTE: the script is written to plot the time series from 1985-2018
#    the years are defined here for plot titles and filenames 
year1,year2 = 1985,2018 
eval_time   = np.arange(year1,year2+1) 

#----
# define simulation
#----
sim = 'A'

#----
# define where to save plots/txt files
#----
save_to_dir ='/global/cfs/cdirs/m4003/cnissen/Plots/RECCAPv2_SO_Paper/Fig_8/'
if not os.path.exists(save_to_dir):
    print ('Created '+save_to_dir)
    os.makedirs(save_to_dir)
    

In [4]:
#-----
# define data sets to consider
#-----
# NOTE: there is 6 different atm. inversions in the provided file, only load the ones that start in 1990 
# NOTE: AOML has to be the first in the list! there is a few places where the exclusion of AOML is 
# hard-coded as "[1:]"

# variable name of interest
var = '2D_CO2'

models     = ('CCSM-WHOI','CESM-ETHZ','CNRM-ESM2-1','EC-Earth3','FESOM_REcoM_HR','FESOM_REcoM_LR',\
             'MOM6-Princeton','MPIOM-HAMOCC','MRI-ESM2-1','NorESM-OC1.2',\
             'ORCA025-GEOMAR','ORCA1-LIM3-PISCES','PlankTOM12','ROMS-SouthernOcean-ETHZ') 
models2     = ('CCSM-WHOI','CESM-ETHZ','CNRM-ESM2-1','EC-Earth3','FESOM_REcoM_HR','FESOM_REcoM_LR',\
             'MOM6-Princeton','MPIOM-HAMOCC','MRI-ESM2-1','NorESM-OC1.2',\
             'ORCA025-GEOMAR','ORCA1-LIM3-PISCES','PlankTOM12','ROMS-SO-ETHZ') 
ind_mpi = models.index("MPIOM-HAMOCC")
ind_not_mpi = [i for i, s in enumerate(models) if 'MPIOM-HAMOCC' not in s]
print ('Index of MPI:',ind_mpi)
print ('All inidces except MPI:',ind_not_mpi)
data_assim  = ('BSOSE','ECCO-Darwin')
data_ocim   = ('OCIM-v2014-CTL','OCIM-v2021')
#data_atminv = ['Atm_inv1','Atm_inv2','Atm_inv4']  #'Atm_inv3','Atm_inv5','Atm_inv6' -> start later than 1990
data_atminv = ['Atm_inv1','Atm_inv2','Atm_inv3','Atm_inv4','Atm_inv5','Atm_inv6'] # for 2015-2018, consider all 6
data_prod   = ('AOML_EXTRAT','CMEMS-LSCE-FFNN','CSIRML6','JenaMLS','JMAMLR',\
             'LDEO-HPD','NIES-ML3','OceanSODAETHZ','SOMFFN') 
data_watson = ['UOEX_Wat20']
soccom      = ('SOCCOM_Jena','SOCCOM_SOMFFN')

versionID_models     = ('20211125','v20211122','v20211208','v20220323','v20211119','v20211119',\
                       'v20220125','v20220110','v20220502','v20211125',\
                       'v20210804','v20211215','v20220404','v20220630') 
versionID_data_assim  = ('I134','v20210712')
versionID_data_ocim   = ('v20210607','v20210511')
versionID_data_atminv = ['v20211008']
versionID_data_prod   = ('v20211130','v20210709','v20211117','v20211126','v20211208',\
                       'v20211210','v20220222','v20211207','v20211121')
versionID_data_watson = ['v20211204']

# list of filenames for data products
filename_data_prod = ('fgco2_AOML_EXTRAT_1997-2020_v20211018.nc4',\
                      'fgco2_CMEMS-LSCE-FFNN_1985-2018_v20210709.nc',\
                      'fgco2_CSIRML6_1985-2018_v20211117.nc',\
                      'fgco2_JenaMLS_1985-2018_v20211126.nc',\
                      'fgco2_JMAMLR_1985-2019_v20211208.nc',\
                      'fgco2_LDEO_HPD_1985-2018_v20211210.nc',\
                      'fgco2_NIES-ML3_1980-2020_v20220222.nc',\
                      'fgco2_OceanSODAETHZ_1985-2018_v20211207.nc',\
                      'fgco2_MPI_SOMFFN_1982-2019_v20211121.nc')
filename_data_watson = ['fgco2_UOEX_Wat20_1985-2019_v20211204.nc']

subregions = ('STSS-Atl','STSS-Ind','STSS-Pac',\
              'SPSS-Atl','SPSS-Ind','SPSS-Pac',\
              'ICE-Atl','ICE-Ind','ICE-Pac','STSS','SPSS','ICE','all')   

print ('Models:',len(models),len(versionID_models))
print ('Data products:',len(data_prod),len(filename_data_prod))
print ('Data Watson2020:',len(data_watson),len(versionID_data_watson))
print ('Data assimilating models:',len(data_assim),len(versionID_data_assim))
print ('Data OCIM:',len(data_ocim),len(versionID_data_assim))
print ('Data ATM inversion:',len(data_atminv),len(versionID_data_atminv))


Index of MPI: 7
All inidces except MPI: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13]
Models: 14 14
Data products: 9 9
Data Watson2020: 1 1
Data assimilating models: 2 2
Data OCIM: 2 2
Data ATM inversion: 6 1


In [5]:
#----
# FUNCTIONS
#----

# transform longitude (e.g., go from -179.5:179.5 to 0.5:359.5)
def transform_lon_coord(data):
    # change lon coordinate in 2D array from 0-360 to -180:180
    # for 2D arrays: assume lon coordinate to be the 2nd dimension  
    # for 3D arrays: assume lon coordinate to be the 3rd dimension  
    if len(data.shape)==2:
        data_transformed          = np.empty_like(data)
        data_transformed[:,0:180] = data[:,180:]
        data_transformed[:,180:]  = data[:,0:180] 
    elif len(data.shape)==3:
        data_transformed          = np.empty_like(data)
        try:
            data_transformed[:,:,0:180] = data[:,:,180:]
            data_transformed[:,:,180:]  = data[:,:,0:180]
        except:
            data_transformed[:,0:180,:] = data[:,180:,:]
            data_transformed[:,180:,:]  = data[:,0:180,:] 
    elif len(data.shape)==4:
        data_transformed          = np.empty_like(data)
        data_transformed[:,0:180,:,:] = data[:,180:,:,:]
        data_transformed[:,180:,:,:]  = data[:,0:180,:,:] 
    elif len(data.shape)==1:
        data_transformed          = np.empty_like(data)
        data_transformed[0:180] = data[180:]
        data_transformed[180:]  = data[0:180]
    return data_transformed


In [6]:
#------
# load RECCAP mask for SO
# make sure region mask matches the submitted products in terms of longitude!
#------

# SO RECCAP regions
#reccap_mask_SO   = Dataset(path_mask+'RECCAP2_region_masks_all_v20221025.nc')
#regions          = reccap_mask_SO_2.variables['southern'][:,:] #"1.SO STSS, 2.SO SPSS, 3.SO ICE"
#lon_regions      = reccap_mask_SO_2.variables['lon'][:]
#print(np.min(lon_regions),np.max(lon_regions)) # lon should be from 0-360 (if it is from -180:180, use transform_lon_coord)
#print(np.min(regions_2),np.max(regions_2))

# SO RECCAP regions
reccap_mask_SO = Dataset(path_mask+'reccap_regions_SOsubs.nc')
regions        = reccap_mask_SO.variables['SO_basins_biomes'][:,:]
lon_regions    = reccap_mask_SO.variables['lon'][:]
print(np.min(lon_regions),np.max(lon_regions)) # lon should be from 0-360 (if it is from -180:180, use transform_lon_coord)

# lon in file is -180:180 -> want 0-360, so transform here
regions = transform_lon_coord(regions)
print(np.min(regions),np.max(regions))


-179.5 179.5
0.0 8.0


In [7]:
#-----
# load data
#-----
      
years_in_file = np.arange(1980,2018+1)

#-----
# atmospheric inversion
#-----
flux_atminv = np.nan*np.ones([180,360,len(eval_time),len(data_atminv)])
for ii in range(0,len(data_atminv)):
    print ('Load '+data_atminv[ii])
    ff = Dataset(path_atminv+'GCP2021_inversions_for_RECCAP2_1x1_version1_1_20211122.nc') 
    # this file contains 6 products!
    data = np.squeeze(ff.variables['ocean_flux_NOT_adjusted'][ii,:,:,:]) # 1990-2020, .mean(axis=0)
    data = data[:-24,:,:] # kick out 2019 & 2020
    data = annualmean(data)
    data = np.transpose(data,[1,2,0]) # time is last
    print (data.shape)
    try: 
        data[data.mask==True]=np.nan
    except: 
        pass
    data[data==0]=np.nan # set land to NaN
    flux_atminv[:,:,5:,ii] = data
    print ('Change sign')
    flux_atminv[:,:,:,ii] = -1*flux_atminv[:,:,:,ii]
    ff.close()
    del data
print ('Transform longitude to 0:360')
flux_atminv = transform_lon_coord(flux_atminv)
    
        
#-----
# models
#-----
flux_models = np.nan*np.ones([180,360,len(eval_time),len(models)])
for ii in range(0,len(models)):
    print ('Load '+models[ii])
    ff = Dataset(path_models+models[ii]+'_'+var+'_'+versionID_models[ii]+'/'+\
                    'fgco2_'+models[ii]+'_'+sim+'_1_gr_1980-2018_'+versionID_models[ii]+'.nc')
    data = np.squeeze(ff.variables['fgco2'][:,:,:])
    data = annualmean(data)
    if models[ii] in ['CCSM-WHOI']: # kick out years 1958-1979
        data = -1*data[22:,:,:] # flip sign
    print (data.shape)
    if models[ii] in ['MPIOM-HAMOCC']: # kick out 2019
        data = data[:-1,:,:]
    try: 
        data[data.mask==True]=np.nan
    except: 
        pass
    data[data==0]=np.nan
        
    # find position into which to write annual means (depends on years provided in submission)
    if models[ii] in ['OCIM-v2014-CTL','CCSM-WHOI']: # these ones stop in 2017
        start_ind = np.where(np.asarray(years_in_file)==year1)[0][0]
        end_ind   = np.argmin(np.abs(eval_time-2017)) 
    else:
        start_ind = np.where(np.asarray(years_in_file)==year1)[0][0]
        end_ind   = np.where(np.asarray(years_in_file)==year2)[0][0]
        
    data = np.transpose(data,[1,2,0])
    if models[ii] in ['OCIM-v2014-CTL','CCSM-WHOI']:
        flux_models[:,:,:end_ind+1,ii] = data[:,:,start_ind:]
    else: 
        flux_models[:,:,:,ii] = data[:,:,start_ind:end_ind+1] #np.nanmean(data,axis=0)
    ff.close()
    del data
    
#-----
# OCIM
#-----
flux_ocim = np.nan*np.ones([180,360,len(eval_time),len(data_ocim)])
for ii in range(0,len(data_ocim)):
    print ('Load '+data_ocim[ii])
    ff = Dataset(path_models+data_ocim[ii]+'_'+var+'_'+versionID_data_ocim[ii]+'/'+\
                    'fgco2_'+data_ocim[ii]+'_'+sim+'_1_gr_1980-2018_'+versionID_data_ocim[ii]+'.nc')
    data = np.squeeze(ff.variables['fgco2'][:,:,:])
    data = annualmean(data)
    try: 
        data[data.mask==True]=np.nan
    except: 
        pass
    # find position into which to write annual means (depends on years provided in submission)
    if data_ocim[ii] in ['OCIM-v2014-CTL']:
        start_ind = np.where(np.asarray(years_in_file)==year1)[0][0]
        end_ind   = np.argmin(np.abs(eval_time-2017)) 
    else:
        start_ind = np.where(np.asarray(years_in_file)==year1)[0][0]
        end_ind   = np.where(np.asarray(years_in_file)==year2)[0][0]
        
    data = np.transpose(data,[1,2,0])
    if data_ocim[ii] in ['OCIM-v2014-CTL']:
        flux_ocim[:,:,:end_ind+1,ii] = data[:,:,start_ind:]
    else: 
        flux_ocim[:,:,:,ii] = data[:,:,start_ind:end_ind+1] #np.nanmean(data,axis=0)
    ff.close()
    del data
        
#-----
# data products
#-----
flux_data_products = np.nan*np.ones([180,360,len(eval_time),len(data_prod)])
for ii in range(0,len(data_prod)):
    print ('Load '+data_prod[ii])
    ff = Dataset(path_data+data_prod[ii]+'_'+versionID_data_prod[ii]+'/'+\
                    filename_data_prod[ii])
    data = np.squeeze(ff.variables['fgco2'][:,:,:])
    if data_prod[ii] in ['AOML_RANDOMF','AOML_EXTRAT']: # get starting time of this product 
        # -> to get rid of incomplete year in beginning
        time = np.squeeze(ff.variables['time'][:])
        day = time[0]
        start = date(1980,1,1)      # This is the "days since" part
        delta = timedelta(day)     # Create a time delta object from the number of days
        offset = start + delta      # Add the specified number of days to 1990
        print(offset)               # >>>  2015-12-01
        del time,day,start,delta,offset
        # delete first 4 entries!
        data = data[4:,:,:]
        #data = data[:-24,:,:] # kick out 2019 & 2020
            
    if data_prod[ii] in ['NIES-ML3']: # reorganize dimensions to math the others
        data = np.transpose(data,[2,1,0])
    if data_prod[ii] in ['SOMFFN']: # reorganize dimensions to math the others
        data = np.transpose(data,[2,1,0])
        data = data[3*12:,:,:] # kick out 1982-1984
    if data_prod[ii] in ['JMAMLR','NIES-ML3']: 
        data = data[5*12:,:,:] # kick out 1980-1984 (filename suggests it starts in 1985, but timedim suggests otherwise)
    if data_prod[ii] in ['JMAMLR','NIES-nn','CSIRML6','SOMFFN']: # 1985-2019
        data = data[:-12,:,:] # kick out 2019
    if data_prod[ii] in ['NIES-ML3']: # 1985-2020
        data = data[:-24,:,:] # kick out 2019 & 2020
    if data_prod[ii] in ['NIES-nn','NIES-ML3']: # mask missing values
        data[data<-9999999]=np.nan
        
    data = annualmean(data)
    print (data.shape)
    data = np.transpose(data,[1,2,0]) # time is last
        
    if data_prod[ii] in ['AOML_RANDOMF','AOML_EXTRAT']: #1997-2020 in file, but 280 months -> 2020 not complete? 
        # it is 1998-2020
        data = data[:,:,:-2] # kick out 2019 & 2020
        years_in_file2 = np.arange(1998,2018+1)
        start_ind = np.where(np.asarray(eval_time)==years_in_file2[0])[0][0]
        flux_data_products[:,:,start_ind:,ii] = data
        del years_in_file2,start_ind
    else: 
        flux_data_products[:,:,:,ii] = data
             
    # for some products, change sign so that pos=into ocean
    if data_prod[ii] in ['NIES-nn']:
        print ('Change sign')
        flux_data_products[:,:,:,ii] = -1*flux_data_products[:,:,:,ii] 
    ff.close()
    del data

#-----
# data Watson2020
#-----
flux_data_watson = np.nan*np.ones([180,360,len(eval_time),len(data_watson)])
for ii in range(0,len(data_watson)):
    print ('Load '+data_watson[ii])
    ff = Dataset(path_data+data_watson[ii]+'_'+versionID_data_watson[ii]+'/'+\
                    filename_data_watson[ii])
    ind_start_watson = 12*3 # skip first three years, i.e. 1985-1987 (wind climatology is used)
    data = np.squeeze(ff.variables['fgco2'][ind_start_watson:,:,:]) 
    data = annualmean(data)
    data = np.transpose(data,[1,2,0]) # time is last
    if data_watson[ii] in ['UOEX_Wat20']: # 1985-2019
        data = data[:,:,:-1]
        ind_start_watson = 3 # here, only annual
        flux_data_watson[:,:,ind_start_watson:,ii] = data
    ff.close()
    del data
     
#-----
# data-assimilating models 
#-----
flux_data_assim = np.nan*np.ones([180,360,len(eval_time),len(data_assim)])
for ii in range(0,len(data_assim)):
    print ('Load '+data_assim[ii])
    ff = Dataset(path_models+data_assim[ii]+'_'+var+'_'+versionID_data_assim[ii]+'/'+\
                    'fgco2_'+data_assim[ii]+'_'+sim+'_1_gr_1980-2018_'+versionID_data_assim[ii]+'.nc')
    data = np.squeeze(ff.variables['fgco2'][:,:,:])
    if data_assim[ii] in ['ECCO-Darwin']: # transform longitudes
        data = transform_lon_coord(data) 
    data = annualmean(data)
    data = np.transpose(data,[1,2,0]) # time is last
    if data_assim[ii] in ['BSOSE']: #2013-2019
        data = data[:,:,:-1] # kick out 2019
        years_in_file2 = np.arange(2013,2018+1)
        start_ind = np.where(np.asarray(eval_time)==years_in_file2[0])[0][0]
        flux_data_assim[:,:,start_ind:,ii] = data
        del years_in_file2,start_ind
    elif data_assim[ii] in ['ECCO-Darwin']: #1985-2018
        start_ind = np.argmin(np.abs(eval_time-1995)) 
        flux_data_assim[:,:,start_ind:,ii] = data[:,:,:]
        del start_ind
    # for some products, change sign so that pos=into ocean
    if data_assim[ii] in ['BSOSE']:
        print ('Change sign')
        flux_data_assim[:,:,:,ii] = -1*flux_data_assim[:,:,:,ii] 
    ff.close()
    del data
    
    

Load Atm_inv1
(180, 360, 29)
Change sign
Load Atm_inv2
(180, 360, 29)
Change sign
Load Atm_inv3
(180, 360, 29)
Change sign
Load Atm_inv4
(180, 360, 29)
Change sign
Load Atm_inv5
(180, 360, 29)
Change sign
Load Atm_inv6
(180, 360, 29)
Change sign
Transform longitude to 0:360
Load CCSM-WHOI
(38, 180, 360)
Load CESM-ETHZ
(39, 180, 360)
Load CNRM-ESM2-1
(39, 180, 360)
Load EC-Earth3
(39, 180, 360)
Load FESOM_REcoM_HR


/global/u2/c/cnissen/scripts_reccap/annualmean.py:43: RuntimeWarning: Mean of empty slice
  var_annualmean[t,:,:] = np.nanmean(datamB, axis=0)


(39, 180, 360)
Load FESOM_REcoM_LR
(39, 180, 360)
Load MOM6-Princeton
(39, 180, 360)
Load MPIOM-HAMOCC
(40, 180, 360)
Load MRI-ESM2-1
(39, 180, 360)
Load NorESM-OC1.2
(39, 180, 360)
Load ORCA025-GEOMAR
(39, 180, 360)
Load ORCA1-LIM3-PISCES
(39, 180, 360)
Load PlankTOM12
(39, 180, 360)
Load ROMS-SouthernOcean-ETHZ
(39, 180, 360)
Load OCIM-v2014-CTL
Load OCIM-v2021
Load AOML_EXTRAT
1997-09-15
(23, 180, 360)
Load CMEMS-LSCE-FFNN
(34, 180, 360)
Load CSIRML6
(34, 180, 360)
Load JenaMLS
(34, 180, 360)
Load JMAMLR
(34, 180, 360)
Load LDEO-HPD
(34, 180, 360)
Load NIES-ML3


/tmp/ipykernel_1528226/243556053.py:109: UserWarning: WARNING: _FillValue not used since it
cannot be safely cast to variable data type
  data = np.squeeze(ff.variables['fgco2'][:,:,:])


(34, 180, 360)
Load OceanSODAETHZ
(34, 180, 360)
Load SOMFFN
(34, 180, 360)
Load UOEX_Wat20
Load BSOSE
Change sign
Load ECCO-Darwin


In [8]:
#----
# load SOCCOM files from Seth
#----

ff = Dataset(path_soccom+'RECCAP_regrid_Jena_CS.nc')
soccom1 = ff.variables['F_CO2'][:] 
time1 = ff.variables['time'][:] # starts in Jan 1957, ends in Dec 2020
ff.close()
ff = Dataset(path_soccom+'RECCAP_regrid_SOM_FFN.nc')
soccom2 = ff.variables['F_CO2'][:]
time2 = ff.variables['time'][:] # starts in Jan 1982, ends in Dec 2019
ff.close()
print (soccom1.shape,soccom2.shape)

start = date(1950,1,1)      # This is the "days since" part
offset1 = start + timedelta(time1[0])      # Add the specified number of days to 1990
offset2 = start + timedelta(time2[0])      # Add the specified number of days to 1990
print(offset1,offset2) 

end1 = start + timedelta(time1[-1])      # Add the specified number of days to 1990
end2 = start + timedelta(time2[-1])      # Add the specified number of days to 1990
print(end1,end2) 

# reduce data to Jan-1985 until Dec-2018
soccom1 = soccom1[(28)*12:-24,:,:]
soccom2 = soccom2[(3)*12:-12,:,:]
print (soccom1.shape,soccom2.shape )

# avg over the chosen season
soccom1 = annualmean(-1*soccom1) # CHANGE SIGN
soccom2 = annualmean(-1*soccom2)

soccom_all = np.stack((soccom1,soccom2))
print(soccom_all.shape)

#np.nan*np.ones([180,360,len(eval_time),len(data_assim)])
flux_soccom = np.transpose(soccom_all,[2,3,1,0])
print (flux_soccom.shape)



(768, 180, 360) (456, 180, 360)
1957-01-15 1982-01-15
2020-12-15 2019-12-15
(408, 180, 360) (408, 180, 360)
(2, 34, 180, 360)
(180, 360, 34, 2)


In [9]:
#------
# print some numbers (to spot any obvious unit problems)
#------

for ii in range(0,len(soccom)):
    print (soccom[ii]+': '+str(np.nanmin(flux_soccom[:,:,:,ii]))+' '+str(np.nanmax(flux_soccom[:,:,:,ii])))
        
for ii in range(0,len(models)):
    print (models[ii]+': '+str(np.nanmin(flux_models[:,:,:,ii]))+' '+str(np.nanmax(flux_models[:,:,:,ii])))

for ii in range(0,len(data_ocim)):
    print (data_ocim[ii]+': '+str(np.nanmin(flux_ocim[:,:,:,ii]))+' '+str(np.nanmax(flux_ocim[:,:,:,ii])))

for ii in range(0,len(data_prod)):
    print (data_prod[ii]+': '+str(np.nanmin(flux_data_products[:,:,:,ii]))+' '+str(np.nanmax(flux_data_products[:,:,:,ii])))

for ii in range(0,len(data_watson)):
    print (data_watson[ii]+': '+str(np.nanmin(flux_data_watson[:,:,:,ii]))+' '+str(np.nanmax(flux_data_watson[:,:,:,ii])))

for ii in range(0,len(data_assim)):
    print (data_assim[ii]+': '+str(np.nanmin(flux_data_assim[:,:,:,ii]))+' '+str(np.nanmax(flux_data_assim[:,:,:,ii])))

for ii in range(0,len(data_atminv)):
    print (data_atminv[ii]+': '+str(np.nanmin(flux_atminv[:,:,:,ii]))+' '+str(np.nanmax(flux_atminv[:,:,:,ii])))


SOCCOM_Jena: -1.9408281820965678e-07 3.408697509806508e-07
SOCCOM_SOMFFN: -1.8549687532292555e-07 2.2637804079616248e-07
CCSM-WHOI: -3.328032391891611e-07 2.3799766779575293e-07
CESM-ETHZ: -1.8722379024893598e-07 1.8758713338229427e-07
CNRM-ESM2-1: -3.473808677055083e-07 7.243008834001572e-07
EC-Earth3: -3.455088517512195e-07 2.9743890195277345e-07
FESOM_REcoM_HR: -1.0287609477898954e-06 3.291037252258339e-07
FESOM_REcoM_LR: -5.16851747625652e-07 3.551971909955131e-07
MOM6-Princeton: -6.316397609341927e-07 8.285434885537167e-07
MPIOM-HAMOCC: -2.1326543730992853e-07 2.601472033347818e-07
MRI-ESM2-1: -6.728191124238947e-07 2.867927548777516e-07
NorESM-OC1.2: -3.108359862835641e-07 3.516230151490163e-07
ORCA025-GEOMAR: -5.994956284336938e-07 3.7668123493913067e-07
ORCA1-LIM3-PISCES: -4.1642874748504255e-07 2.7747370268116356e-07
PlankTOM12: -4.364177641491551e-07 1.7493054826900334e-07
ROMS-SouthernOcean-ETHZ: -2.626861999033281e-07 4.0850900973055104e-07
OCIM-v2014-CTL: -5.43813831818880

In [10]:
#------
# get surface area of all the biomes
#------
# calculate area with sw.dist

def get_areas_biomes(area,regions,subregions):
    # calculate area-weighted averages of a given quantity in data according to biomes defined
    # after Fay & McKinley (2014), use RECCAP mask loaded further up
    # provide "area" (with lon from 0:360)
    
    # only consider area for SO RECCAP mask
    area[regions.mask==True]=0
    print ('Total Southern Ocean ocean surface area '+str(np.nansum(area))+' m2')
    
    data_avg = np.zeros(13) # 9 regions in SO, + 3 for combined sectors, +1 for whole SO
    for i in range(0,13):
        reg2  = regions.ravel()
        area2 = area.ravel()
        if subregions[i] in ['STSS-Atl']:
            ind_region = np.where(reg2==0)[0]
        elif subregions[i] in ['STSS-Ind']:
            ind_region = np.where(reg2==3)[0]
        elif subregions[i] in ['STSS-Pac']:
            ind_region = np.where(reg2==6)[0]
        elif subregions[i] in ['SPSS-Atl']:
            ind_region = np.where(reg2==1)[0]
        elif subregions[i] in ['SPSS-Ind']:
            ind_region = np.where(reg2==4)[0]
        elif subregions[i] in ['SPSS-Pac']:
            ind_region = np.where(reg2==7)[0]
        elif subregions[i] in ['ICE-Atl']:
            ind_region = np.where(reg2==2)[0]
        elif subregions[i] in ['ICE-Ind']:
            ind_region = np.where(reg2==5)[0]
        elif subregions[i] in ['ICE-Pac']:
            ind_region = np.where(reg2==8)[0]
        elif subregions[i] in ['STSS']:
            ind_region = np.where((reg2==0) | (reg2==3) | (reg2==6))[0]
        elif subregions[i] in ['SPSS']:
            ind_region = np.where((reg2==1) | (reg2==4) | (reg2==7))[0]
        elif subregions[i] in ['ICE']:
            ind_region =np.where((reg2==2) | (reg2==5) | (reg2==8))[0]
        elif subregions[i] in ['all']:
            ind_region = np.where(reg2>=0)[0]
        data_avg[i] = np.nansum(area2[ind_region]) #np.nansum(data2[ind_region]*area2[ind_region]/total_area)
        del reg2, area2,ind_region
    return data_avg

# define for filtering, but don't use for area calculation
lon = np.arange(0.5,359.5+1,1)
lat = np.arange(-89.5,89.5+1,1)

# need the box boundaries for area calculation
xi = np.arange(0,360+1,1)
yi = np.arange(-90,90+1,1) 

area = np.zeros((len(xi)-1,len(yi)-1))
#calculate area
for i in range(0,len(xi)-1): #laenge pruefen!
    if i==np.round(len(xi)/2): 
        print ('50% done')
    for j in range(0,len(yi)-1): 
        dist1 = sw.dist([yi[j],yi[j+1]],[xi[i],xi[i]])
        dist2 = sw.dist([yi[j],yi[j]],[xi[i],xi[i+1]])
        area[i][j] = float(dist1[0]) * float(dist2[0]) *1000 *1000 #m2
area = area.transpose()
print ('Global area:',np.sum(area),'m2') 
area_global = np.copy(area)

area_biomes = get_areas_biomes(area,regions,subregions)

#print (area_biomes)
print('TEST: the following three should be identical')
print (np.sum(area_biomes[0:9])) # 9 subregions
print (np.sum(area_biomes[9:12])) # 3 subregions
print (np.sum(area_biomes[12])) # 1 subregion 


50% done
Global area: 509364377985322.94 m2
Total Southern Ocean ocean surface area 77409028462219.4 m2
TEST: the following three should be identical
77409028462219.4
77409028462219.42
77409028462219.42


In [11]:
#-----
# get subareas avg
#-----
# NOTE: for each product/model, get areas of biomes separately
# the areas are not the same for the different prodcuts

def get_subarea_avg(flux_models,regions,area,eval_time,models_ABCD,subregions):
    flux_subareas  = np.nan*np.ones([len(eval_time),len(models_ABCD),len(subregions)]) 
    biome_areas    = np.nan*np.ones([len(models_ABCD),len(subregions)]) 
    for pp in range(0,len(models_ABCD)):
        counter=0 # only for printing of total area
        print ('Process '+models_ABCD[pp])
        for i in range(0,len(subregions)):
            for yy in range(0,len(eval_time)):
                data1 = flux_models[:,:,yy,pp]
                reg2  = regions.ravel()  # SO RECCAP mask
                area2 = area.ravel() # surface area
                data2 = data1.ravel()
                if subregions[i] in ['STSS-Atl']:
                    ind_region = np.where(reg2==0)[0]
                elif subregions[i] in ['STSS-Ind']:
                    ind_region = np.where(reg2==3)[0]
                elif subregions[i] in ['STSS-Pac']:
                    ind_region = np.where(reg2==6)[0]
                elif subregions[i] in ['SPSS-Atl']:
                    ind_region = np.where(reg2==1)[0]
                elif subregions[i] in ['SPSS-Ind']:
                    ind_region = np.where(reg2==4)[0]
                elif subregions[i] in ['SPSS-Pac']:
                    ind_region = np.where(reg2==7)[0]
                elif subregions[i] in ['ICE-Atl']:
                    ind_region = np.where(reg2==2)[0]
                elif subregions[i] in ['ICE-Ind']:
                    ind_region = np.where(reg2==5)[0]
                elif subregions[i] in ['ICE-Pac']:
                    ind_region = np.where(reg2==8)[0]
                elif subregions[i] in ['STSS']:
                    ind_region = np.where((reg2==0) | (reg2==3) | (reg2==6))[0]
                elif subregions[i] in ['SPSS']:
                    ind_region = np.where((reg2==1) | (reg2==4) | (reg2==7))[0]
                elif subregions[i] in ['ICE']:
                    ind_region =np.where((reg2==2) | (reg2==5) | (reg2==8))[0]
                elif subregions[i] in ['all']:
                    ind_region = np.where(reg2>=0)[0]
                    
                ind_not_nan_data = np.where(~np.isnan(data2[ind_region]))[0] # only consider data points that are filled
                total_area = np.sum(area2[ind_region][ind_not_nan_data])
                #total_area = np.sum(area2[ind_region])
                if np.nansum(data1)!=0: # only for years which are filled
                    if (subregions[i] in ['all']) & (counter==0):
                        print (total_area )
                        counter = 1
                    biome_areas[pp,i] = total_area 
                    flux_subareas[yy,pp,i] = np.nansum(data2[ind_region]*area2[ind_region]/total_area)
                del data1,reg2,area2,data2,ind_region,total_area
    return flux_subareas,biome_areas
    

In [12]:
#-----
# get subareas avg: data products
#-----
# NOTE: for each product/model, get areas of biomes separately
# the areas are not the same for the different prodcuts
def get_subarea_avg_TEST(flux_models,regions,area,eval_time,models_ABCD,subregions):
    biome_areas    = np.nan*np.ones([len(eval_time),len(models_ABCD),len(subregions)]) 
    for pp in range(0,len(models_ABCD)):
        counter=0 # only for printing of total area
        print ('Process '+models_ABCD[pp])
        for i in range(0,len(subregions)):
            for yy in range(0,len(eval_time)):
                data1 = flux_models[:,:,yy,pp]
                reg2  = regions.ravel()  # SO RECCAP mask
                area2 = area.ravel() # surface area
                data2 = data1.ravel()
                if subregions[i] in ['STSS-Atl']:
                    ind_region = np.where(reg2==0)[0]
                elif subregions[i] in ['STSS-Ind']:
                    ind_region = np.where(reg2==3)[0]
                elif subregions[i] in ['STSS-Pac']:
                    ind_region = np.where(reg2==6)[0]
                elif subregions[i] in ['SPSS-Atl']:
                    ind_region = np.where(reg2==1)[0]
                elif subregions[i] in ['SPSS-Ind']:
                    ind_region = np.where(reg2==4)[0]
                elif subregions[i] in ['SPSS-Pac']:
                    ind_region = np.where(reg2==7)[0]
                elif subregions[i] in ['ICE-Atl']:
                    ind_region = np.where(reg2==2)[0]
                elif subregions[i] in ['ICE-Ind']:
                    ind_region = np.where(reg2==5)[0]
                elif subregions[i] in ['ICE-Pac']:
                    ind_region = np.where(reg2==8)[0]
                elif subregions[i] in ['STSS']:
                    ind_region = np.where((reg2==0) | (reg2==3) | (reg2==6))[0]
                elif subregions[i] in ['SPSS']:
                    ind_region = np.where((reg2==1) | (reg2==4) | (reg2==7))[0]
                elif subregions[i] in ['ICE']:
                    ind_region =np.where((reg2==2) | (reg2==5) | (reg2==8))[0]
                elif subregions[i] in ['all']:
                    ind_region = np.where(reg2>=0)[0]

                ind_not_nan_data = np.where(~np.isnan(data2[ind_region]))[0] # only consider data points that are filled
                total_area = np.sum(area2[ind_region][ind_not_nan_data])
                #total_area = np.sum(area2[ind_region])
                if np.nansum(data1)!=0: # only for years which are filled
                    if (subregions[i] in ['all']) & (counter==0):
                        print (total_area )
                        counter = 1
                    biome_areas[yy,pp,i] = total_area 
                del data1,reg2,area2,data2,ind_region,total_area
    return biome_areas

print ('-----')
print ('data products')
print ('-----')
biome_area_data_products_varying = get_subarea_avg_TEST(flux_data_products,regions,area,\
                                                    eval_time,data_prod,subregions)



-----
data products
-----
Process AOML_EXTRAT
77182686197228.12
Process CMEMS-LSCE-FFNN
73647074238260.6
Process CSIRML6
75956069483462.88
Process JenaMLS
77405832654098.36
Process JMAMLR
72001325799725.03
Process LDEO-HPD
77043201248352.36
Process NIES-ML3
76336956437489.22
Process OceanSODAETHZ
76482430896315.38
Process SOMFFN
74444375365152.88


In [13]:
#------
# get subarea mean of models, data product, data assimilation products
#------

# calculate subarea averages
#subregions = ('STSS_Atl','SPSS_Atl','ICE_Atl','STSS_Ind','SPSS_Ind',\
#              'ICE_Ind','STSS_Pac','SPSS_Pac','ICE_Pac','STSS','SPSS','ICE')#,'all')
# 13 regions: 9 regions in SO, + 3 for combined sectors, +1 for whole SO

print ('-----')
print ('SOCCOM')
print ('-----')
flux_soccom_subareas,biome_area_soccom = get_subarea_avg(flux_soccom,regions,area,\
                                                eval_time,soccom,subregions)
    
print ('-----')
print ('models')
print ('-----')
flux_models_subareas,biome_area_models = get_subarea_avg(flux_models,regions,area,\
                                                eval_time,models,subregions)
print ('-----')
print ('OCIM')
print ('-----')
flux_ocim_subareas,biome_area_ocim = get_subarea_avg(flux_ocim,regions,area,\
                                                eval_time,data_ocim,subregions)
        
print ('-----')
print ('data products')
print ('-----')
flux_data_products_subareas,biome_area_data_products = get_subarea_avg(flux_data_products,regions,area,\
                                                eval_time,data_prod,subregions)

print ('-----')
print ('data products/Watson2020')
print ('-----')
flux_data_watson_subareas,biome_area_data_watson = get_subarea_avg(flux_data_watson,regions,area,\
                                                eval_time,data_watson,subregions)

print ('-----')
print ('data-assimilating models')
print ('-----')
flux_data_assim_subareas,biome_area_data_assim = get_subarea_avg(flux_data_assim,regions,area,\
                                                eval_time,data_assim,subregions)

print ('-----')
print ('atm inversions')
print ('-----')
flux_atminv_subareas,biome_area_atminv = get_subarea_avg(flux_atminv,regions,area,\
                                                eval_time,data_atminv,subregions)
    

-----
SOCCOM
-----
Process SOCCOM_Jena
76893579241079.84
Process SOCCOM_SOMFFN
74444375365152.88
-----
models
-----
Process CCSM-WHOI
74492275264407.1
Process CESM-ETHZ
76149904481959.67
Process CNRM-ESM2-1
77371852756049.22
Process EC-Earth3
77367548050317.14
Process FESOM_REcoM_HR
77075005194060.84
Process FESOM_REcoM_LR
77075005194060.84
Process MOM6-Princeton
77075005194060.84
Process MPIOM-HAMOCC
76003285884316.31
Process MRI-ESM2-1
77325948214566.8
Process NorESM-OC1.2
77374505093557.94
Process ORCA025-GEOMAR
77289199561810.75
Process ORCA1-LIM3-PISCES
77402636845977.3
Process PlankTOM12
76670454088793.06
Process ROMS-SouthernOcean-ETHZ
76894536194319.66
-----
OCIM
-----
Process OCIM-v2014-CTL
77075005194060.84
Process OCIM-v2021
77075005194060.84
-----
data products
-----
Process AOML_EXTRAT
77182686197228.12
Process CMEMS-LSCE-FFNN
73647074238260.6
Process CSIRML6
75956069483462.88
Process JenaMLS
77405832654098.36
Process JMAMLR
72001325799725.03
Process LDEO-HPD
7704320124835

In [14]:
#--- 
# print some numbers
#---

ss = -1

print ('######')
print ('Subregion '+subregions[ss]+':')
print ('######')

for ii in range(0,len(soccom)):
        print (soccom[ii]+': '+str(flux_soccom_subareas[-2,ii,ss]))
                
for ii in range(0,len(models)):
        if models[ii] in ['OCIM-v2014-CTL','CCSM-WHOI']: # year 2018 not available!, print year 2017!
             print (models[ii]+': '+str(flux_models_subareas[-2,ii,ss])) # last year, whole SO 
        else:
            print (models[ii]+': '+str(flux_models_subareas[-1,ii,ss])) # last year, whole SO 
            
for ii in range(0,len(data_ocim)):
        if data_ocim[ii] in ['OCIM-v2014-CTL']: # year 2018 not available!, print year 2017!
             print (data_ocim[ii]+': '+str(flux_ocim_subareas[-2,ii,ss])) # last year, whole SO 
        else:
            print (data_ocim[ii]+': '+str(flux_ocim_subareas[-1,ii,ss])) # last year, whole SO 

for ii in range(0,len(data_prod)):
        print (data_prod[ii]+': '+str(flux_data_products_subareas[-1,ii,ss]))# last year, whole SO 
    
for ii in range(0,len(data_watson)):
        print (data_watson[ii]+': '+str(flux_data_watson_subareas[-1,ii,ss]))
        
for ii in range(0,len(data_assim)):
        print (data_assim[ii]+': '+str(flux_data_assim_subareas[-1,ii,ss]))# last year, whole SO 

for ii in range(0,len(data_atminv)):
        print (data_atminv[ii]+': '+str(flux_atminv_subareas[-1,ii,ss]))# last year, whole SO 


        

######
Subregion all:
######
SOCCOM_Jena: 1.5839072879459185e-08
SOCCOM_SOMFFN: 2.197143579812024e-08
CCSM-WHOI: 2.035881093091796e-08
CESM-ETHZ: 3.120886545477298e-08
CNRM-ESM2-1: 1.842900355708883e-08
EC-Earth3: 1.5997418995185397e-08
FESOM_REcoM_HR: 4.683412766973994e-08
FESOM_REcoM_LR: 3.871533671451623e-08
MOM6-Princeton: 3.9561852176573295e-08
MPIOM-HAMOCC: 4.1949297658834164e-08
MRI-ESM2-1: 3.443743117206353e-08
NorESM-OC1.2: 4.3581461856629926e-08
ORCA025-GEOMAR: 4.581613858215737e-08
ORCA1-LIM3-PISCES: 2.985258511197887e-08
PlankTOM12: 2.95879817034716e-08
ROMS-SouthernOcean-ETHZ: 3.5841547686978557e-08
OCIM-v2014-CTL: 5.340414076981168e-08
OCIM-v2021: 5.37501090560348e-08
AOML_EXTRAT: 4.166614991545029e-08
CMEMS-LSCE-FFNN: 3.955209097029648e-08
CSIRML6: 3.4891743188601984e-08
JenaMLS: 3.750394673667549e-08
JMAMLR: 4.2990999302987564e-08
LDEO-HPD: 2.9882843521484294e-08
NIES-ML3: 2.9262687758036157e-08
OceanSODAETHZ: 3.632073182837807e-08
SOMFFN: 2.9514761965870207e-08
UOEX_Wa

In [15]:
#-----
# get multi-model and multi-data mean
#-----

print (flux_models_subareas.shape)
print (flux_ocim_subareas.shape)
print (flux_data_products_subareas.shape)
print (flux_data_watson_subareas.shape)
print (flux_data_assim_subareas.shape)

ind_list = [0,1,3] # only include inv1, inv2, inv4 here -> only these start in 1990
print (flux_atminv_subareas[:,ind_list,:].shape)

multi_model_mean      = np.nanmean(flux_models_subareas,axis=1)
multi_ocim_mean       = np.nanmean(flux_ocim_subareas,axis=1)
multi_data_prod_mean  = np.nanmean(flux_data_products_subareas,axis=1)
multi_data_watson_mean= np.nanmean(flux_data_watson_subareas,axis=1)
multi_data_assim_mean = np.nanmean(flux_data_assim_subareas,axis=1)
multi_atminv_mean     = np.nanmean(flux_atminv_subareas[:,ind_list,:],axis=1) # only include those that start in 1990
multi_soccom_mean     = np.nanmean(flux_soccom_subareas,axis=1)

#multi_data_prod_mean  = np.nanmean(flux_data_products[:,:,1:],axis=2) # leave out AOML_EXTRAT (starts later)
    
print (data_prod)
print ('AOML (all):',flux_data_products_subareas[:,0,-1])
print ('CCSM-WHOI (all):',flux_models_subareas[:,0,-1])
    
# get mean area for each product class
#print biome_area_atminv.shape
area_mean_models        = np.mean(biome_area_models[:,:],axis=0) 
area_mean_data_products = np.mean(biome_area_data_products[:,:],axis=0) 
area_mean_watson        = np.mean(biome_area_data_watson[:,:],axis=0)
area_mean_data_assim    = np.mean(biome_area_data_assim[:,:],axis=0)
area_mean_ocim          = np.mean(biome_area_ocim[:,:],axis=0)
area_mean_atminv        = np.mean(biome_area_atminv[ind_list,:],axis=0) # only include those that start in 1990
area_mean_soccom        = np.mean(biome_area_soccom[:,:],axis=0)

print ('area_mean_models (all):',area_mean_models[-1])
print ('area_mean_data_products (all):',area_mean_data_products[-1])
print ('area_mean_watson (all):',area_mean_watson[-1])
print ('area_mean_data_assim (all):',area_mean_data_assim[-1])
print ('area_mean_ocim (all):',area_mean_ocim[-1])
print ('area_mean_atminv (all):',area_mean_atminv[-1])
print ('area_mean_soccom (all):',area_mean_soccom[-1])


(34, 14, 13)
(34, 2, 13)
(34, 9, 13)
(34, 1, 13)
(34, 2, 13)
(34, 3, 13)
('AOML_EXTRAT', 'CMEMS-LSCE-FFNN', 'CSIRML6', 'JenaMLS', 'JMAMLR', 'LDEO-HPD', 'NIES-ML3', 'OceanSODAETHZ', 'SOMFFN')
AOML (all): [           nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan 1.40615694e-08 1.52688026e-08 1.65644182e-08
 1.63762934e-08 1.97764252e-08 2.42505902e-08 2.51150112e-08
 2.91051833e-08 2.72589266e-08 2.46552996e-08 2.61350050e-08
 3.06001597e-08 3.55619885e-08 3.86187106e-08 3.47239339e-08
 3.52279716e-08 3.66856423e-08 3.68788035e-08 3.77060392e-08
 3.90201101e-08 4.16661499e-08]
CCSM-WHOI (all): [8.59687351e-09 9.92957096e-09 8.48138119e-09 1.18567546e-08
 7.60841138e-09 1.14097375e-08 1.27120112e-08 1.71361824e-08
 1.07665565e-08 1.11882030e-08 1.22975347e-08 1.36141723e-08
 1.02441266e-08 1.02380263e-08 9.89095468e-09 1.25992091e-08
 1.20049642e-08 

/tmp/ipykernel_1528226/1738166859.py:17: RuntimeWarning: Mean of empty slice
  multi_data_watson_mean= np.nanmean(flux_data_watson_subareas,axis=1)
/tmp/ipykernel_1528226/1738166859.py:18: RuntimeWarning: Mean of empty slice
  multi_data_assim_mean = np.nanmean(flux_data_assim_subareas,axis=1)
/tmp/ipykernel_1528226/1738166859.py:19: RuntimeWarning: Mean of empty slice
  multi_atminv_mean     = np.nanmean(flux_atminv_subareas[:,ind_list,:],axis=1) # only include those that start in 1990


In [16]:
#----
# load river file, apply to model fields
#----

river_adjustment = True
if river_adjustment: 
    river_adj_names = ('Lacroix2020')
    ff = Dataset(path_river+'fgco2_lacroix-river_v20211223.nc')
    river = ff.variables['fgco2'][:] # lon is 0-360, needs to be transformed for plotting
    ff.close()
    print (river.shape)

subregions = ('STSS-Atl','STSS-Ind','STSS-Pac',\
              'SPSS-Atl','SPSS-Ind','SPSS-Pac',\
              'ICE-Atl','ICE-Ind','ICE-Pac','STSS','SPSS','ICE','all')

# 13 regions: 9 regions in SO, + 3 for combined sectors, +1 for whole SO

if river_adjustment: 
    #------
    # models
    #------
    river_subareas   = np.nan*np.ones([len(subregions)]) 
    biome_area_river      = np.nan*np.ones([len(subregions)]) 
    print ('Process river file ')
    for i in range(0,len(subregions)):
        data1 = np.copy(river)
        reg2  = regions.ravel()  # SO RECCAP mask
        area2 = area.ravel() # surface area
        data2 = data1.ravel()
            
        if subregions[i] in ['STSS-Atl']:
            ind_region = np.where(reg2==0)[0]
        elif subregions[i] in ['STSS-Ind']:
            ind_region = np.where(reg2==3)[0]
        elif subregions[i] in ['STSS-Pac']:
            ind_region = np.where(reg2==6)[0]
        elif subregions[i] in ['SPSS-Atl']:
            ind_region = np.where(reg2==1)[0]
        elif subregions[i] in ['SPSS-Ind']:
            ind_region = np.where(reg2==4)[0]
        elif subregions[i] in ['SPSS-Pac']:
            ind_region = np.where(reg2==7)[0]
        elif subregions[i] in ['ICE-Atl']:
            ind_region = np.where(reg2==2)[0]
        elif subregions[i] in ['ICE-Ind']:
            ind_region = np.where(reg2==5)[0]
        elif subregions[i] in ['ICE-Pac']:
            ind_region = np.where(reg2==8)[0]
        elif subregions[i] in ['STSS']:
            ind_region = np.where((reg2==0) | (reg2==3) | (reg2==6))[0]
        elif subregions[i] in ['SPSS']:
            ind_region = np.where((reg2==1) | (reg2==4) | (reg2==7))[0]
        elif subregions[i] in ['ICE']:
            ind_region =np.where((reg2==2) | (reg2==5) | (reg2==8))[0]
        elif subregions[i] in ['all']:
            ind_region = np.where(reg2>=0)[0]
        ind_not_nan_data = np.where(~np.isnan(data2[ind_region]))[0] # only consider data points that are filled
        total_area = np.sum(area2[ind_region][ind_not_nan_data])
        if subregions[i] in ['all']:
            print (total_area)
        biome_area_river[i] = total_area 
        #total_area = np.sum(area2[ind_region])
        if np.nansum(data1)!=0: # only for years which are filled
            river_subareas[i] = np.nansum(data2[ind_region]*area2[ind_region]/total_area)
        del data1,reg2,area2,data2,ind_region,total_area
        
#-------
# save field to apply to model output
#-------

if river_adjustment:
    add_to_models = np.copy(river_subareas)
else:
    add_to_models = np.zeros_like(river_subareas) # add a field of zeros
print ('River fluxes to add for subregion all:',add_to_models[-1])

    
#----
# check for river fluxes
#----
if river_adjustment:
    print ('')
    print ('River flux adjustment (pos=outgassing):')
    factor_river = 365.25*86400.*12.011/1e15 # don't use "-1" here; global number should positive = outgassing due to rivers
    for ss in range(0,len(subregions)):
        print (subregions[ss]+': '+str(factor_river*river_subareas[ss]*biome_area_river[ss])+' Pg C yr-1')
    # check: add up 3 subregions
    print ('based on STSS/SPSS/ICE: '+str(np.sum(factor_river*river_subareas[9:12]*biome_area_river[9:12]))+' Pg C yr-1')

    print ('Global ocean area: '+str(np.sum(area_global))+' m2')
    print ('Global river flux: '+str(np.nansum(factor_river*np.multiply(river,area_global)))+' Pg C yr-1')
    print ('SO/Global: '+str(100*factor_river*river_subareas[-1]*biome_area_river[-1]/np.nansum(factor_river*np.multiply(river,area_global)))+'%')
    print ('')
    #print river


    

(180, 360)
Process river file 
76118473462222.84
River fluxes to add for subregion all: 1.3662420531327198e-09

River flux adjustment (pos=outgassing):
STSS-Atl: -0.01546799211558836 Pg C yr-1
STSS-Ind: 0.0026367052644284345 Pg C yr-1
STSS-Pac: -0.0013755711900539146 Pg C yr-1
SPSS-Atl: -0.008262337193420477 Pg C yr-1
SPSS-Ind: 0.045257119271516585 Pg C yr-1
SPSS-Pac: 0.016270984555087874 Pg C yr-1
ICE-Atl: -0.0035661499261086895 Pg C yr-1
ICE-Ind: 0.0054982502435766975 Pg C yr-1
ICE-Pac: -0.0015724400214355222 Pg C yr-1
STSS: -0.014206858041213835 Pg C yr-1
SPSS: 0.05326576663318398 Pg C yr-1
ICE: 0.00035966029603248497 Pg C yr-1
all: 0.03941856888800264 Pg C yr-1
based on STSS/SPSS/ICE: 0.03941856888800263 Pg C yr-1
Global ocean area: 509364377985322.94 m2
Global river flux: 0.6158395517429716 Pg C yr-1
SO/Global: 6.400785525456877%



In [17]:
#-----
# get trends
#-----

In [22]:
#----
# FUNCTIONS to get drift-corrected trends
#----

def get_trend_MMM_MDM_w_simB(biome_area_models,biome_area_data_products,models_all,data_all,\
                             add_to_models,biome_area_river,models,data_prod,\
                                               eval_time,factor,unit,slope,year_start,year_end):
    # slope -> trend in simB over chosen year_start and year_end
    
    factor_river = 365.25*86400.*12.011/1e15 # for river fluxes, no need to multiply by "-1" -> pos=outgassing already!

    #-----
    # MODELS
    #-----
    add_rivers = factor_river*biome_area_river*add_to_models
    add_each_year = add_rivers*np.ones(len(eval_time))
    
    # create array of area: models x time (area constant in time)
    area_2d = np.transpose(np.tile(biome_area_models[:],[len(eval_time),1])) # should be models x time
    
    # apply model trends
    models_all2 = np.zeros_like(models_all)
    # "slope" is already in PgC yr-1 yr-1, 
    # make sure that when applying it to fluxes, the latter are already in Pg C yr-1
    for mm in range(0,len(models)):
        for yy in range(0,len(eval_time)):
                models_all2[mm,yy] = area_2d[mm,yy]*factor*models_all[mm,yy]-slope[mm]
                
    # correcting with trend from year_start to year_end
    models_all2a = np.zeros_like(models_all)
    # "slope" is already in PgC yr-1 yr-1, 
    # make sure that when applying it to fluxes, the latter are already in Pg C yr-1
    for mm in range(0,len(models)):
        for yy in range(0,len(eval_time)):
                models_all2a[mm,yy] = area_2d[mm,yy]*factor*models_all[mm,yy]-slope[mm]
                
    # for calculation of multi-model mean, use the array that is drift-corrected
    model_data_mean = np.nanmean(models_all2[:,:],axis=0)+add_each_year
    
    #-----
    # DATA -> area not constant in time, passed as an input argument 
    #-----
    obs_data_mean  = np.nanmean(biome_area_data_products[:,:]*factor*data_all[:,:],axis=0)
      
    #---
    # plot linear fits (1985-2000)
    #---
    
    #year_start,year_end = 1985,2000
    ind2000 = np.where((eval_time>=year_start) & (eval_time<=year_end))[0]
    x = np.array(np.arange(0,len(eval_time[ind2000]))).reshape((-1, 1))
    
    # MODELS
    a1 = model_data_mean[ind2000]
    ind_noNaN = np.where(~np.isnan(a1))[0]
    a1 = a1[ind_noNaN]
    x1 = x[ind_noNaN]
    model = LinearRegression()
    model.fit(x1, a1)
    model = LinearRegression().fit(x, a1)
    slope_models1 = model.coef_[0]
    # Make predictions using the testing set
    pred_data = model.predict(np.array(np.arange(0,len(eval_time[ind2000]))).reshape((-1, 1)))
    
    # DATA PRODUCTS
    a1 = obs_data_mean[ind2000]
    ind_noNaN = np.where(~np.isnan(a1))[0]
    a1 = a1[ind_noNaN]
    x1 = x[ind_noNaN]
    model = LinearRegression()
    model.fit(x1, a1)
    model = LinearRegression().fit(x, a1)
    slope_data1 = model.coef_[0]
    
    # Make predictions using the testing set
    pred_data = model.predict(np.array(np.arange(0,len(eval_time[ind2000]))).reshape((-1, 1)))
    
    # get trend in each model (to get spread)
    # if end year is 2018, don't use CCSM (ends in 2017)
    trend_models1 = np.nan*np.ones(len(models))
    for mm in range(0,len(models)):
        if year_end==2018:
            if not models[mm] in ['CCSM-WHOI']:
                a1 = models_all2a[mm,ind2000] # updated here: use models corrected with trend in simB from 1985-2000
                ind_noNaN = np.where(~np.isnan(a1))[0]
                a1 = a1[ind_noNaN]
                x1 = x[ind_noNaN]
                model = LinearRegression()
                model.fit(x1, a1)
                model = LinearRegression().fit(x, a1)
                #print 'slope in '+models[mm]+' (per decade): '+str(10*model.coef_[0])
                trend_models1[mm] = model.coef_[0] # store in array
                del a1,ind_noNaN,x1,model
        else:
            a1 = models_all2a[mm,ind2000] # updated here: use models corrected with trend in simB from 1985-2000
            ind_noNaN = np.where(~np.isnan(a1))[0]
            a1 = a1[ind_noNaN]
            x1 = x[ind_noNaN]
            model = LinearRegression()
            model.fit(x1, a1)
            model = LinearRegression().fit(x, a1)
            #print 'slope in '+models[mm]+' (per decade): '+str(10*model.coef_[0])
            trend_models1[mm] = model.coef_[0] # store in array
            del a1,ind_noNaN,x1,model
    
    # get trend in each data product (to get spread)
    trend_data1 = np.nan*np.ones(len(data_prod))
    for mm in range(1,len(data_prod)): # exclude AOML
        a1 = biome_area_data_products[mm,ind2000]*factor*data_all[mm,ind2000]
        ind_noNaN = np.where(~np.isnan(a1))[0]
        a1 = a1[ind_noNaN]
        x1 = x[ind_noNaN]
        model = LinearRegression()
        model.fit(x1, a1)
        model = LinearRegression().fit(x, a1)
        #print 'slope in '+data_prod[mm]+' (per decade): '+str(10*model.coef_[0])
        trend_data1[mm] = model.coef_[0] # store in array
        del a1,ind_noNaN,x1,model 
    
    round2 = 1000
    
    #print('Linear trends in Pg C yr-1 dec-1: ')
    print('GOBMs, '+str(year_start)+'-'+str(year_end)+': '+\
                    str(np.round(round2*10*np.nanmean(trend_models1))/round2)+' +/- '+\
                    str(np.round(round2*10*np.nanstd(trend_models1))/round2))
    print('pCO2 products, '+str(year_start)+'-'+str(year_end)+': '+\
                    str(np.round(round2*10*slope_data1)/round2)+' +/- '+\
                    str(np.round(round2*10*np.nanstd(trend_data1))/round2))
    

In [23]:
#------
# get trends 
#-------

# CHECK: the list below should match the one at the very top of this script
models_short = ('CCSM_WHOI','CESM_ETHZ','CNRM_ESM2_1','EC_Earth3','FESOM_REcoM_HR','FESOM_REcoM_LR',\
             'MOM6_Princeton','MPIOM_HAMOCC','MRI_ESM2_1','NorESM_OC1.2',\
             'ORCA025_GEOMAR','ORCA1_LIM3_PISCES','PlankTOM12','ROMS-SouthernOcean-ETHZ')

# trend is in Pg C yr-1 dec-1

# -> divide by 10 to get Pg C yr-1 yr-1
# -> subtract this from flux of each year in sim A -> DOUBLE-CHECK THE SIGN!!!!

unit    = 'FCO$_{2}$ (Pg C yr$^{-1}$)'
factor2 = -1*365.25*86400.*12.011/1e15 # conversion factor from mol C m-2 s-1 to PgC yr-1

# transpose arrays for function:
# expected input: data: models x regions x time, data_mean: regions x time; swap dimensions in arrays
flux_models_subareas_plot     = np.transpose(flux_models_subareas,[1,2,0])
flux_data_products_plot       = np.transpose(flux_data_products_subareas,[1,2,0])
flux_data_assim_subareas_plot = np.transpose(flux_data_assim_subareas,[1,2,0])
flux_data_ocim_subareas_plot  = np.transpose(flux_ocim_subareas,[1,2,0])
flux_atminv_subareas_plot  = np.transpose(flux_atminv_subareas,[1,2,0])
#multi_model_mean_plot = np.mean(flux_models_subareas_plot,axis=0)
multi_model_mean_plot      = np.transpose(multi_model_mean,[1,0])
multi_data_prod_mean_plot  = np.transpose(multi_data_prod_mean,[1,0])
multi_data_assim_mean_plot = np.transpose(multi_data_assim_mean,[1,0])
multi_data_watson_mean_plot = np.transpose(multi_data_watson_mean,[1,0])
multi_atminv_mean_plot = np.transpose(multi_atminv_mean,[1,0])
multi_soccom_mean_plot = np.transpose(multi_soccom_mean,[1,0])
multi_soccom_mean_plot[:,0:-4] = np.nan # only show years 2015-2018 for SOCCOM products


#----
# get trends for chosen year_start and year_end
#----

year_start,year_end = 1985,2000

# which years do I want?
# 1985-2000 IN PAPER
# 1986-2000
# 1987-2000

# 1985-1999
# 1985-2001
# 1985-2002

# 2001-2018 IN PAPER
# 1999-2018
# 2002-2018
# 2003-2018

# 2001-2017
# 2001-2016

year_start_list = [1985,1986,1987,1985,1985,1985]
year_end_list   = [2000,2000,2000,1999,2001,2002]
print('Linear trends in Pg C yr-1 dec-1: ')
    
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()
    
    rr = subregions.index("all") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
print()
print('#########')
print() 
#---
# ICE
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()
    
    rr = subregions.index("ICE") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
print()
print('#########')
print()
#---
# SPSS
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()
    
    rr = subregions.index("SPSS") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
    
print()
print('#########')
print()
#---
# STSS
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()
    
    rr = subregions.index("STSS") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
    


Linear trends in Pg C yr-1 dec-1: 

all, 1985-2000
GOBMs, 1985-2000: -0.085 +/- 0.068
pCO2 products, 1985-2000: 0.005 +/- 0.124
GOBMs, 1986-2000: -0.076 +/- 0.066
pCO2 products, 1986-2000: 0.025 +/- 0.13
GOBMs, 1987-2000: -0.072 +/- 0.062
pCO2 products, 1987-2000: 0.047 +/- 0.139
GOBMs, 1985-1999: -0.08 +/- 0.069
pCO2 products, 1985-1999: -0.004 +/- 0.123
GOBMs, 1985-2001: -0.082 +/- 0.063
pCO2 products, 1985-2001: 0.023 +/- 0.125
GOBMs, 1985-2002: -0.1 +/- 0.066
pCO2 products, 1985-2002: 0.016 +/- 0.124

#########


ICE, 1985-2000
GOBMs, 1985-2000: -0.029 +/- 0.023
pCO2 products, 1985-2000: -0.025 +/- 0.042
GOBMs, 1986-2000: -0.029 +/- 0.02
pCO2 products, 1986-2000: -0.025 +/- 0.044
GOBMs, 1987-2000: -0.024 +/- 0.017
pCO2 products, 1987-2000: -0.019 +/- 0.045
GOBMs, 1985-1999: -0.03 +/- 0.026
pCO2 products, 1985-1999: -0.024 +/- 0.038
GOBMs, 1985-2001: -0.029 +/- 0.022
pCO2 products, 1985-2001: -0.019 +/- 0.04
GOBMs, 1985-2002: -0.03 +/- 0.02
pCO2 products, 1985-2002: -0.022 +/- 0.041

In [24]:

# 2001-2018 IN PAPER
# 1999-2018
# 2002-2018
# 2003-2018

# 2001-2017
# 2001-2016

year_start_list = [2001,1999,2000,2002,2003,2001,2001]
year_end_list   = [2018,2018,2018,2018,2018,2017,2016]
print('Linear trends in Pg C yr-1 dec-1: ')
    
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()

    rr = subregions.index("all") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
    
print()
print('#########')
print() 
#---
# ICE
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()

    rr = subregions.index("ICE") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
print()
print('#########')
print()
#---
# SPSS
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()

    rr = subregions.index("SPSS") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
    
print()
print('#########')
print()
#---
# STSS
#---
for yy in range(0,len(year_start_list)):
    year_start,year_end = year_start_list[yy],year_end_list[yy]
    #----
    # load trends for simB
    #----
    f1 = Dataset(path_trend+'LinearTrend_'+str(year_start)+'_'+str(year_end)+'_CO2_flux_simB.nc')
    slope_models_simB = np.zeros([len(models),len(subregions)])
    for mm in range(0,len(models)):
        slope_models_simB[mm,:] = f1.variables['trend_fgco2_'+models_short[mm]][:]/10
    f1.close()

    rr = subregions.index("STSS") # region index
    if yy==0:
        print ('')   
        print (subregions[rr]+', '+str(year_start)+'-'+str(year_end))
    get_trend_MMM_MDM_w_simB(biome_area_models[:,rr],np.transpose(biome_area_data_products_varying[:,:,rr]),\
                                                    flux_models_subareas_plot[:,rr,:],\
                                                    flux_data_products_plot[:,rr,:],\
                                                    add_to_models[rr],biome_area_river[rr],\
                                                    models,data_prod,eval_time,factor2,unit,slope_models_simB[:,rr],year_start,year_end)
    

    

Linear trends in Pg C yr-1 dec-1: 

all, 2001-2018
GOBMs, 2001-2018: -0.113 +/- 0.028
pCO2 products, 2001-2018: -0.258 +/- 0.059
GOBMs, 1999-2018: -0.127 +/- 0.03
pCO2 products, 1999-2018: -0.255 +/- 0.062
GOBMs, 2000-2018: -0.118 +/- 0.03
pCO2 products, 2000-2018: -0.257 +/- 0.06
GOBMs, 2002-2018: -0.097 +/- 0.033
pCO2 products, 2002-2018: -0.241 +/- 0.058
GOBMs, 2003-2018: -0.105 +/- 0.034
pCO2 products, 2003-2018: -0.239 +/- 0.048
GOBMs, 2001-2017: -0.111 +/- 0.03
pCO2 products, 2001-2017: -0.262 +/- 0.072
GOBMs, 2001-2016: -0.108 +/- 0.03
pCO2 products, 2001-2016: -0.27 +/- 0.082

#########


ICE, 2001-2018
GOBMs, 2001-2018: -0.035 +/- 0.015
pCO2 products, 2001-2018: -0.064 +/- 0.038
GOBMs, 1999-2018: -0.036 +/- 0.015
pCO2 products, 1999-2018: -0.065 +/- 0.039
GOBMs, 2000-2018: -0.036 +/- 0.015
pCO2 products, 2000-2018: -0.063 +/- 0.037
GOBMs, 2002-2018: -0.033 +/- 0.016
pCO2 products, 2002-2018: -0.056 +/- 0.035
GOBMs, 2003-2018: -0.032 +/- 0.016
pCO2 products, 2003-2018: -0.054 +